In [1]:
import logging
import os

import numpy as np

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
logging.getLogger().setLevel(logging.ERROR)
import tensorflow as tf
tf.get_logger().setLevel(logging.ERROR)

import sys
sys.path.append("../src")

from toolz import keyfilter

from Evaluation import run_ensembles, load_models
from LoadData import CurrentDatasets
from Helpers import create_confusion_matrix_plot_from_csv

In [2]:
dataset_names = [dataset.value for dataset in CurrentDatasets]

# in a nested list all elements on the same level get equal weights
# eg [foo, [bar, bar, bar, [baz, baz]]] is weighted [1/2, [1/8, 1/8, 1/8, [1/16, 1/16]]]

ensembles = {"All": ["Encoder", "FCN", "MCDCNN", "MLP", "Resnet", "Time_CNN"],
             "Best3": ["Encoder", "Resnet", "MLP"],
             "Resnet10": [f"Resnet-{number}" for number in range(0,10)],
             "NNE": [[f"Resnet-{number}" for number in range(0,10)], "FCN", "Encoder"]}
csv_name = '../results/test_Ensembles.csv'

In [3]:
used_ensembles=ensembles #keyfilter(lambda ensemble: ensemble in ["Resnet10"], ensembles)

In [4]:
run_ensembles(dataset_names=dataset_names, ensembles=used_ensembles, verbose=True).to_csv(csv_name)

1/11:	cricket_y
('Encoder', 'FCN', 'MCDCNN', 'MLP', 'Resnet', 'Time_CNN')
(0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666)


AttributeError: 'str' object has no attribute 'removesuffix'

In [ ]:
create_confusion_matrix_plot_from_csv(csv_name, verbose=True)